In [ ]:
# All you need for this is -

# pip install opencv-python

In [72]:
import cv2

import numpy as np

from sklearn.metrics import pairwise

In [73]:
cv2.__version__

'4.1.0'

In [63]:
background = None

accumulated_weight = 0.55

roi_top = 10
roi_bottom = 300
roi_right = 300
roi_left = 600

## Part 1 - This function gives us the accumulated weights.

In [64]:
def calc_accum_avg(frame, accumulated_weight):
    
    global background
    
    # This will just return none if there is nothing in background.
    if background is None:
        background = frame.copy().astype('float')
        return None
    
    # If the background has some value then this function will return the running average of the detected frame.
    cv2.accumulateWeighted(frame, background, accumulated_weight)   

## Part 2 - The next step is to use Thresholding to grab the hand segment from the ROI.

In [65]:
def segment(frame, threshold_min = 25):
    global background
    
    # This is used to calc the abs difference of the frame and background
    diff = cv2.absdiff(background.astype('uint8'), frame)
    
    # It calculates the threshold using the absolute difference and the min threshold and max.
    _ , thresholded = cv2.threshold(diff, threshold_min, 255, cv2.THRESH_BINARY)
    
    # This finds the contours using the thresholded image above. We take the external contours.
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # When the contour is 0 it returns nothing.
    if len(contours) == 0:
        return None
    
    else:
        
        # When the largest external contour detected in the ROI, is the hand, we store it in hand_segment. 
        hand_segment = max(contours, key = cv2.contourArea)
        
        # Returns the thresholded image with hand segment (contour of the hand).
        return(thresholded, hand_segment)

## Part 3 - Finger Counting with Convex Hull 

In [66]:
# A convex hull draws the polygon by connecting around the most external points in the frame. 

def count_fingers(thresholded, hand_segment):
    
    ''' After casting the thresholded and hand_segment we create this convex hull '''
    conv_hull = cv2.convexHull(hand_segment)
    
    ''' Calculating the most extreme 4 points '''
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    ''' Calculating the Center '''
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    ''' Calculating the distance from center to all the extreme points '''
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]
    
    ''' Calculating the max Distance'''
    max_distance = distance.max()
    
    ''' Creating a circle out of that max distance'''
    radius = int(0.8 * max_distance) 
    circumference = (2 * np.pi * radius) 
    
    ''' Creatign a circular region of interest '''
    circular_roi = np.zeros(thresholded.shape[:2], dtype = 'uint8')
    
    ''' Draw the circular ROI '''
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10)
    
    ''' Creating a Mask for the circle roi '''
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask = circular_roi)
    
    ''' Using that circle we put up contours on everything thats outside of the certain region of the circle '''
    contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    ''' We start with finger count = 0 '''
    count = 0
    
    ''' We start counting with some limitations such as out of the wrist and not some noise '''
    # For every contour in contours list
    for cnt in contours:
        # We take the boundary box
        (x, y, w, h) = cv2.boundingRect(cnt)
        
        # Making sure that it is not the wrist
        out_of_wrist = ((cY + (cY * 0.25)) > (y + h))
        
        # Making sure that we are not including any noise way outside of the circle.
        limit_points = ((circumference * 0.25) > cnt.shape[0])
        
        if out_of_wrist and limit_points:
            count += 1
            
    return count   # Here we return the count of the fingers. 

## Part 4 - Bringing it together.

In [69]:
cam = cv2.VideoCapture(0)

# Intialize a frame count
num_frames = 0

# keep looping, until interrupted
while True:
    # get the current frame
    ret, frame = cam.read()

    # flip the frame so that it is not the mirror view
    frame = cv2.flip(frame, 1)

    # clone the frame
    frame_copy = frame.copy()

    # Grab the ROI from the frame
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]

    # Apply grayscale and blur to ROI
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    # For the first 30 frames we will calculate the average of the background.
    # We will tell the user while this is happening
    if num_frames < 60:
        calc_accum_avg(gray, accumulated_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, "WAIT! GETTING BACKGROUND AVG.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow("Finger Count",frame_copy)
            
    else:
        # now that we have the background, we can segment the hand.
        
        # segment the hand region
        hand = segment(gray)

        # First check if we were able to actually detect a hand
        if hand is not None:
            
            # unpack
            thresholded, hand_segment = hand

            # Draw contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0),1)

            # Count the fingers
            fingers = count_fingers(thresholded, hand_segment)

            # Display count
            cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

            # Also display the thresholded image
            cv2.imshow("Thesholded", thresholded)

    # Draw ROI Rectangle on frame copy
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)

    # increment the number of frames for tracking
    num_frames += 1

    # Display the frame with segmented hand
    cv2.imshow("Finger Count", frame_copy)


    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()